In [1]:
#pip install gazpacho

In [2]:
from gazpacho import get, Soup
import numpy
import pandas
import requests, bs4
import re, os
import time

In [3]:
# Verifying that the page request is valid
from gazpacho import get, Soup

url = 'https://www.imperialyeast.com/yeast-strains/house'

# fetch step
html = get(url)

# extract step
soup = Soup(html)
soup

<!DOCTYPE html><html class="scroll-smooth"><head><meta charSet="utf-8"/><link rel="icon" href="/favicon.svg"/><meta name="viewport" content="initial-scale=1.0, width=device-width"/><title>House - Imperial Yeast</title><link rel="preload" as="image" imageSrcSet="/_next/image?url=https%3A%2F%2Fimperial-yeast-cms.mur.io%2Fassets%2F178dcd9f-0f59-4670-8af6-b1b48f467e5b%2Fpatterns_ales.png&amp;w=256&amp;q=75 256w, /_next/image?url=https%3A%2F%2Fimperial-yeast-cms.mur.io%2Fassets%2F178dcd9f-0f59-4670-8af6-b1b48f467e5b%2Fpatterns_ales.png&amp;w=384&amp;q=75 384w, /_next/image?url=https%3A%2F%2Fimperial-yeast-cms.mur.io%2Fassets%2F178dcd9f-0f59-4670-8af6-b1b48f467e5b%2Fpatterns_ales.png&amp;w=600&amp;q=75 600w, /_next/image?url=https%3A%2F%2Fimperial-yeast-cms.mur.io%2Fassets%2F178dcd9f-0f59-4670-8af6-b1b48f467e5b%2Fpatterns_ales.png&amp;w=768&amp;q=75 768w, /_next/image?url=https%3A%2F%2Fimperial-yeast-cms.mur.io%2Fassets%2F178dcd9f-0f59-4670-8af6-b1b48f467e5b%2Fpatterns_ales.png&amp;w=900&amp

Use the inspect features in a web browser to view the html tag
![title](data/inspect.png)

In [4]:
# Trial and error to figure out where to pull the html code from
strain_name = soup.find('h1', {'class' : "hero-step text-h1 -md:text-h1-mobile"})
strain_name

<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">House</h1>

### Pull The Strain Name

In [5]:
# Pull The Strain Name

def pull_strain_name(soup):
    strain_name = soup.find('h1', {'class' : "hero-step text-h1 -md:text-h1-mobile"})
    x = str(strain_name)
    print(x)
    start = x.find('mobile')
    end = x.find('</h1>')
    strain = x[start+8:end]
    # print(strain)
    return strain

In [6]:
pull_strain_name(soup)

<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">House</h1>


'House'

### Pull The Strain Number

In [7]:
# use find() to parse a specific HTML
x = soup.find('div', {'class' : 'hero-step animation__delay-short relative'})
x

<div class="pre-animated hero-step animation__delay-short relative float-right top-28 py-20 px-24 bg-grey w-min text-white text-center rounded ml-5 mb-60 -md:hidden">
  <h2>A01</h2>
  <svg width="15" height="16" fill="none" class="mx-auto my-5 fill-gold -md:hidden">
    <path d="M7.44.918L.38 8l7.06 7.082L14.5 8 7.44.918z" fill="inherit"/>
  </svg>
  <div class="space-y-5">
    <p class="link">Versatile</p>
    <p class="link">Neutral</p>
    <p class="link">English</p>
  </div>
</div>

In [8]:
# Pull The Strain Number

def pull_strain_number(soup):
    strain_number = soup.find('div', {'class':'hero-step animation__delay-short relative'})
    x = str(strain_number)
    start = x.find('<h2>')
    number = x[start+4:start+7]
    # print(number)
    return number

In [9]:
pull_strain_number(soup)

'A01'

### Pull The Description

In [10]:
# Pull and save the description

def pull_strain_description(soup):
    description = soup.find('p', {'class':'hero-step animation__delay-long mt-2.5'})
    #print(description)
    x= str(description)
    start = x.find('2.5')
    start = start + 5
    end = x.find('</p>')
    description = x[start:end]
    # print(description)
    return description

In [11]:
pull_strain_description(soup)

'The best of both worlds, House is clean and allows malt and hops to shine. This strain is extremely versatile and flocculent enough to drop out of the beer quickly. Best strain choice for any and all English- inspired recipes, however, does extremely well in hopped up American styles too. House is clean at cold temperatures with increased esters as fermentation temperatures rise.'

### Get Stats on Strain

In [12]:
def pull_stats(soup):
    raw = soup.find('div', {'class':'step col-span-4 grid grid-cols-2 -sm:grid-cols-1 -sm:col-span-1 gap-x-5'})
    x= str(raw)
    #       print(x)
    fs = x.find('Flocculation') + 51
    fe = x.find('Flocculation') + 59
    flocculation = x[fs:fe]
    # print(flocculation)

    attens = x.find('Attenuation') + 49
    attene = x.find('Attenuation') + 59
    attenuation = x[attens:attene]
    # print(attenuation)

    abve = x.find('Alcohol Tolerance') + 56
    abvs = x.find('Alcohol Tolerance') + 59
    abv = x[abve:abvs]
    # print(abv)

    temps = x.find('Temperature</span>') + 48
    tempe = x.find('Temperature</span>') + 60
    temp = x[temps:tempe]
    # print(temp)

    stas = x.find('Diastatic') +48
    stae = x.find('Diastatic') +60
    STA1 = x[stas:stae]
    # print(temp)
    
    
    return(flocculation, attenuation, abv, temp, STA1)

In [13]:
pull_stats(soup)

('high</h6', '>73-75%</h', '10%', '">62-70 °F /', 'negative</h6')

### Pull all the urls

In [14]:
import pandas as pd
urls = pd.read_csv('data/imperial_strain_list.csv')
urls

,Address
0,https://www.imperialyeast.com/yeast-strains/ba...
1,https://www.imperialyeast.com/yeast-strains/ba...
2,https://www.imperialyeast.com/yeast-strains/br...
3,https://www.imperialyeast.com/yeast-strains/br...
4,https://www.imperialyeast.com/yeast-strains/br...
5,https://www.imperialyeast.com/yeast-strains/br...
6,https://www.imperialyeast.com/yeast-strains/bu...
7,https://www.imperialyeast.com/yeast-strains/ca...
8,https://www.imperialyeast.com/yeast-strains/capri
9,https://www.imperialyeast.com/yeast-strains/ch...


In [15]:
url = urls.Address.values.tolist()
url

['https://www.imperialyeast.com/yeast-strains/barbarian',
 'https://www.imperialyeast.com/yeast-strains/bartleby',
 'https://www.imperialyeast.com/yeast-strains/brett-brux',
 'https://www.imperialyeast.com/yeast-strains/brett-clausenii',
 'https://www.imperialyeast.com/yeast-strains/brett-drei',
 'https://www.imperialyeast.com/yeast-strains/brett-lambicus',
 'https://www.imperialyeast.com/yeast-strains/bubbles',
 'https://www.imperialyeast.com/yeast-strains/cablecar',
 'https://www.imperialyeast.com/yeast-strains/capri',
 'https://www.imperialyeast.com/yeast-strains/champion',
 'https://www.imperialyeast.com/yeast-strains/citrus',
 'https://www.imperialyeast.com/yeast-strains/corporate',
 'https://www.imperialyeast.com/yeast-strains/darkness',
 'https://www.imperialyeast.com/yeast-strains/dieter',
 'https://www.imperialyeast.com/yeast-strains/dry-hop',
 'https://www.imperialyeast.com/yeast-strains/fillibuster',
 'https://www.imperialyeast.com/yeast-strains/flagship',
 'https://www.impe

In [16]:
import pandas as pd

def loadTable(soup):
    strain = pull_strain_name(soup)
    #pullMetrics(soup)
    number = pull_strain_number(soup)
    flocculation, attenuation, abv, temp, STA1 = pull_stats(soup)
    description = pull_strain_description(soup)
    entry = {'Strain': [strain],
             'Number' : [number],
        'Attenuation' : [attenuation],
        'Flocculation' : [flocculation],
        'Alcohol_Tolerance' : [abv],
        'Fermentation_Temperature' : [temp],
        'STA1' : [STA1],
        'Description' : [description]}
    df = pd.DataFrame(entry)
    return df

In [17]:
loadTable(soup)

<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">House</h1>


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,House,A01,>73-75%</h,high</h6,10%,""">62-70 °F /",negative</h6,"The best of both worlds, House is clean and al..."


In [18]:
df = loadTable(soup)
df1 = df
df1

<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">House</h1>


,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,House,A01,>73-75%</h,high</h6,10%,""">62-70 °F /",negative</h6,"The best of both worlds, House is clean and al..."


In [19]:
for i in url:
  print(i)
  # fetch step
  html = get(i)

  # extract step
  soup = Soup(html)
  df = loadTable(soup)
  df1 = pd.concat([df1, df])

https://www.imperialyeast.com/yeast-strains/barbarian
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Barbarian</h1>
https://www.imperialyeast.com/yeast-strains/bartleby
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Bartleby</h1>
https://www.imperialyeast.com/yeast-strains/brett-brux
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Brett Brux</h1>
https://www.imperialyeast.com/yeast-strains/brett-clausenii
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Brett Clausenii</h1>
https://www.imperialyeast.com/yeast-strains/brett-drei
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Brett Drei</h1>
https://www.imperialyeast.com/yeast-strains/brett-lambicus
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Brett Lambicus</h1>
https://www.imperialyeast.com/yeast-strains/bubbles
<h1 class="pre-animated hero-step text-h1 -md:text-h1-mobile">Bubbles</h1>
https://www.imperialyeast.com/yeast-strains/cablecar
<h1 class=

In [20]:
df1

,Strain,Number,Attenuation,Flocculation,Alcohol_Tolerance,Fermentation_Temperature,STA1,Description
0,House,A01,>73-75%</h,high</h6,10%,""">62-70 °F /",negative</h6,"The best of both worlds, House is clean and al..."
0,Barbarian,A04,>73-74%</h,medium</,10%,""">62-70 °F /",negative</h6,Commonly referred to as the &quot;conan&quot; ...
0,Bartleby,A46,>75-85%</h,high</h6,10%,""">68-98 °F /",negative</h6,Bartleby is a hornindal Kveik strain that is a...
0,Brett Brux,W13,>55-85%</h,low</h6>,12%,""">64-74 °F /",negative</h6,Brux is a classic Brettanomyces bruxellensis s...
0,Brett Clausenii,W12,>50-60%</h,low</h6>,12%,""">70-85 °F /",negative</h6,Claussenii is a Brettanomyces strain favored f...
0,Brett Drei,W05,>56-85%</h,low</h6>,10%,""">70-85 °F /",positive</h6,A strain of Brettanomyces bruxellensis known t...
0,Brett Lambicus,W10,>55-85%</h,low</h6>,12%,""">70-85 °F /",negative</h6,Lambicus is a Brettanomyces bruxellensis strai...
0,Bubbles,A40,>85-100%</,medium</,10%,""">57-77 °F /",negative</h6,"The clean profile of the yeast, especially whe..."
0,Cablecar,L05,>71-75%</h,medium-h,10%,""">55-65 °F /",negative</h6,Cablecar can produce clean pseudo lagers at al...
0,Capri,I22,>68-74%</h,medium</,10%,""">68-86 °F /",negative</h6,Capri is a small-celled warrior fit to bring o...


# Pulling all the urls that are needed

In [22]:
df1.to_csv('imperial_2024.csv')